# Basic Model usage

In [82]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Optional

# Load environment variables from .env file
load_dotenv()

True

In [83]:
prompt = ChatPromptTemplate.from_template(
    "I need to compute the square root of {num_1} first, then add {num_2}." 
)
model = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=150)

chain = prompt | model | StrOutputParser()
output = chain.invoke({"num_1": "729", "num_2": "21"})
print(output)

To solve this, first compute the square root of 729. The square root of 729 is 27. Then, add 21 to this result:

\[ 27 + 21 = 48. \]

So, the final result is 48.


In [84]:
# LCEL

class CityRecommendationOutput(BaseModel):
    """Information about a city recommendation."""
    description: str = Field(description="citys description")
    dishes: List[str] = Field(description="citys main dishes")
    attractions: List[str] = Field(description="citys main attractions")
    

prompt = ChatPromptTemplate.from_template(
    """
    You are a tour guide in {city}. Please read carefully the following tasks:
    
    Task:
        1. Generate a description about {city} no longer than 10 words
        2. Get favority dishes from {city}
        3. Get main attractions in {city}
    """
)


extraction_functions = [convert_pydantic_to_openai_function(CityRecommendationOutput)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "CityRecommendationOutput"})

chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [78]:
output = chain.invoke({"city": "medellin"})
print(output)
print(type(output))

{'description': 'Vibrant city with rich culture and stunning landscapes.', 'dishes': ['Bandeja Paisa', 'Arepas', 'Empanadas', 'Mondongo', 'Sancocho'], 'attractions': ['Plaza Botero', 'Medellin Metrocable', 'Comuna 13', 'Parque Arvi', 'Museo de Antioquia']}
<class 'dict'>


In [86]:
output = chain.invoke({"city": "rio de janeiro"})
print(output)
print(type(output))

{'description': 'Vibrant city with stunning beaches and lively culture.', 'dishes': ['Feijoada', 'Moqueca', 'Pão de Queijo', 'Brigadeiro'], 'attractions': ['Christ the Redeemer', 'Sugarloaf Mountain', 'Copacabana Beach', 'Ipanema Beach', 'Tijuca National Park']}
<class 'dict'>


In [87]:
prompt = ChatPromptTemplate.from_template(
    "tell me how much is the price of a flight for tomorrow from {from_city} to {to_city} in COP" 
)
chain = prompt | model | StrOutputParser()
output = chain.invoke({"from_city": "medellin", "to_city": "rio de janeiro"})
print(output)

I can't provide real-time data or current prices for flights. To find the most accurate and up-to-date information on flight prices from Medellín to Rio de Janeiro, I recommend checking airline websites, travel booking platforms like Expedia, Kayak, or Google Flights, or contacting a travel agent. Prices can vary based on the airline, time of booking, and availability, so it's best to compare options across different platforms.


In [89]:
prompt = ChatPromptTemplate.from_template(
    "what is the weather like now in {city}" 
)
chain = prompt | model | StrOutputParser()

for city in ["medellin", "rio de janeiro"]:
    output = chain.invoke({"city": city})
    print(output)

I'm unable to provide real-time weather updates. To get the current weather in Medellín, I recommend checking a reliable weather website or using a weather app.
I'm unable to provide real-time weather updates. To get the current weather in Rio de Janeiro, I recommend checking a reliable weather website or app, such as the Weather Channel, BBC Weather, or a local news station's website.
